In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.optim import Adam
import datetime
import torch
import os
from torch.utils.tensorboard import SummaryWriter

In [8]:
device = torch.device('cuda:1')

if torch.cuda.is_available():
    print("Available CUDA devices:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

Available CUDA devices: 2
Device 0: NVIDIA GeForce RTX 4090
Device 1: NVIDIA GeForce RTX 4090


In [11]:
inputdir='../data/'
# Load data
train_data = pd.read_csv(inputdir+'loop_sensor_train.csv')
test_data_x = pd.read_csv(inputdir+'loop_sensor_test_x.csv')

# Convert time to datetime
train_data['t_1h'] = pd.to_datetime(train_data['t_1h'])
test_data_x['t_1h'] = pd.to_datetime(test_data_x['t_1h'])

# Preprocess and split data for year 2023
def preprocess_data(data):
    train = data
    test = []
    for name, group in data[data['t_1h'].dt.year == 2023].groupby(pd.Grouper(key='t_1h', freq='29h')):
        train = pd.concat([train, group.iloc[:24]])
        if len(group) > 24:
            test.append(group.iloc[24])
    return train, pd.DataFrame(test)

train, eval_data = preprocess_data(train_data)
print(train.shape)

(23139649, 4)


In [10]:
# inputdir='../data/'
# # Load data
# train_data = pd.read_csv(inputdir+'loop_sensor_train.csv')
# test_data_x = pd.read_csv(inputdir+'loop_sensor_test_x.csv')

# # Convert time to datetime
# train_data['t_1h'] = pd.to_datetime(train_data['t_1h'])
# test_data_x['t_1h'] = pd.to_datetime(test_data_x['t_1h'])

# # Filter data by year
# train_2022 = train_data[train_data['t_1h'].dt.year == 2022]
# train_2023 = train_data[train_data['t_1h'].dt.year == 2023]

# # Preprocess and split data for year 2023
# def preprocess_data(data):
#     train = []
#     test = []
#     # We need to handle each 29-hour block: 24h for train, 1h for test, 4h discard
#     grouped = data.groupby(pd.Grouper(key='t_1h', freq='29h'))
#     for _, group in grouped:
#         # Add the first 24 hours to train, if available
#         if len(group) >= 24:
#             train.append(group.iloc[:24])
#         # Add the 25th hour to test, if available
#         if len(group) > 24:
#             test.append(group.iloc[24])
#         # We automatically discard the next 4 hours by not including them

#     train = pd.concat(train, ignore_index=True)
#     test = pd.DataFrame(test, columns=data.columns)  # Ensure the DataFrame is properly formatted
#     return train, test

# # Combine the 2022 data with processed 2023 data
# train_2023, test_2023 = preprocess_data(train_2023)
# train_combined = pd.concat([train_2022, train_2023], ignore_index=True)
# test_combined = pd.concat([test_data_x, test_2023], ignore_index=True)

# print(train_combined.shape)


(12571565, 4)


In [12]:
# Function to encode categorical data into one-hot vectors
def encode_categorical(data, column_name, num_classes):
    categories = torch.tensor(data[column_name].values)
    return torch.nn.functional.one_hot(categories, num_classes=num_classes).float()

def data_to_tensor(data, train=1):
    # Convert 't_1h' from datetime to a float representing hours since the start of the dataset
    ref_time = pd.Timestamp('2022-01-01 00:00:00')  # Adjust the reference time as needed
    data['t_1h'] = (data['t_1h'] - ref_time).dt.total_seconds() / 3600.0
    # One-hot encode 'etat_barre'
    etat_barre_encoded = encode_categorical(data, 'etat_barre', 4)
    # Convert to tensors
    iu_ac = torch.tensor(data['iu_ac'].values).unsqueeze(1).float()
    t_1h = torch.tensor(data['t_1h'].values).unsqueeze(1).float()
    # Combine features into a single tensor
    features = torch.cat((iu_ac, t_1h, etat_barre_encoded), dim=1)
    
    if train:
        targets = torch.tensor(data['q'].values).float()
        return TensorDataset(features, targets)
    return TensorDataset(features)

# Assuming 'train', 'eval_data', and 'test_data_x' are already loaded and preprocessed
train_dataset = data_to_tensor(train)
eval_dataset = data_to_tensor(eval_data)
test_dataset_x = data_to_tensor(test_data_x, train=0)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=2048, shuffle=False)
test_loader = DataLoader(test_dataset_x, batch_size=2048, shuffle=False)


RuntimeError: Class values must be smaller than num_classes.

In [ ]:
# # Define LSTM model
# class LSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers):
#         super(LSTMModel, self).__init__()
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
#         self.regressor = nn.Linear(hidden_dim, 1)

#     def forward(self, x):
#         _, (hn, _) = self.lstm(x)
#         out = self.regressor(hn.squeeze(0))
#         return out

# model = LSTMModel(input_dim=3, hidden_dim=50, num_layers=2)
# model.to(device)
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout=0.5):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.regressor = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.regressor(hn.squeeze(0))
        return out

# Example with more hidden units and layers
model = LSTMModel(input_dim=5, hidden_dim=500, num_layers=3, dropout=0.5)
model.to(device)

# Loss and optimizer
criterion = nn.L1Loss()
optimizer = Adam(model.parameters(), lr=0.001)

In [15]:
def train_model(model, train_loader, criterion, optimizer, num_epochs, log_interval=10, checkpoint_dir='./logs'):
    # Move model to GPU
    device = torch.device('cuda:1')
    model.to(device)
    
    # Initialize TensorBoard writer
    writer = SummaryWriter()
    start_epoch = 1
    flag = True
    checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pth')  # specify the filename for checkpoint
    
    # Ensure checkpoint directory exists
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    # Check if checkpoint exists
    if os.path.exists(checkpoint_path):
        print("Loading checkpoint...")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1  # start from next epoch
        
    model.train()
    for epoch in range(start_epoch, num_epochs + start_epoch):
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            if flag:
                print(next(model.parameters()).is_cuda)  
                print(inputs.is_cuda)                    
                flag = False
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            # Log loss to TensorBoard
            if batch_idx % log_interval == 0:
                writer.add_scalar('Loss/train', loss.item(), epoch * len(train_loader) + batch_idx)
            
            print(f'Epoch {epoch}, Batch {batch_idx+1}, Loss: {loss.item()}')
            # Save checkpoint periodically
            if batch_idx % 100 == 0:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss.item(),
                }, checkpoint_path)
    # Close the TensorBoard writer
    writer.close()

# Example usage (assuming model, train_loader, criterion, optimizer are defined)
train_model(model, train_loader, criterion, optimizer, num_epochs=1)

Loading checkpoint...
True
True
Epoch 4, Batch 1, Loss: 520.0232543945312
Epoch 4, Batch 2, Loss: 526.5364379882812
Epoch 4, Batch 3, Loss: 522.0922241210938


/media/ssd4t/anaconda3/envs/traffic/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([4096])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 4, Batch 4, Loss: 495.80145263671875
Epoch 4, Batch 5, Loss: 502.6297302246094
Epoch 4, Batch 6, Loss: 506.8919372558594
Epoch 4, Batch 7, Loss: 487.9410095214844
Epoch 4, Batch 8, Loss: 498.7703857421875
Epoch 4, Batch 9, Loss: 501.1858825683594
Epoch 4, Batch 10, Loss: 521.5285034179688
Epoch 4, Batch 11, Loss: 514.3475952148438
Epoch 4, Batch 12, Loss: 523.0536499023438
Epoch 4, Batch 13, Loss: 487.61322021484375
Epoch 4, Batch 14, Loss: 541.3139038085938
Epoch 4, Batch 15, Loss: 522.6865234375
Epoch 4, Batch 16, Loss: 555.7149047851562
Epoch 4, Batch 17, Loss: 554.218017578125
Epoch 4, Batch 18, Loss: 534.621826171875
Epoch 4, Batch 19, Loss: 521.9705810546875
Epoch 4, Batch 20, Loss: 533.052734375
Epoch 4, Batch 21, Loss: 526.8267822265625
Epoch 4, Batch 22, Loss: 537.8046875
Epoch 4, Batch 23, Loss: 523.1343994140625
Epoch 4, Batch 24, Loss: 533.2232055664062
Epoch 4, Batch 25, Loss: 523.561279296875
Epoch 4, Batch 26, Loss: 531.1241455078125
Epoch 4, Batch 27, Loss: 510.52

/media/ssd4t/anaconda3/envs/traffic/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([701])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [ ]:
# Evaluation
def evaluate_model(model, eval_loader):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for inputs, targets in eval_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
        print(f'Evaluation Loss: {total_loss / len(eval_loader)}')

evaluate_model(model, eval_loader)


In [32]:
# Predict
model.eval()
predictions = []
with torch.no_grad():
    for features, in test_loader:
        features = features.to(device)
        outputs = model(features)
        predictions.extend(outputs.cpu().numpy().flatten())

# Create a new DataFrame with only predictions, set index starting from 1
predictions_df = pd.DataFrame(predictions, columns=['predicted_q'])
predictions_df.index = predictions_df.index + 1  # Adjust index to start from 1

# Save the predictions to a CSV file
predictions_df.to_csv(inputdir+'predictions_only.csv', index_label='ID')

print("Predictions saved to 'predictions_only.csv', with IDs starting from 1.")


Predictions saved to 'predictions_only.csv', with IDs starting from 1.


In [ ]:
# Load the baseline data
baseline_data = pd.read_csv(inputdir+'loop_sensor_test_baseline.csv')

# Prepare test data
test_data_x['t_1h'] = pd.to_datetime(test_data_x['t_1h'])
ref_time = pd.Timestamp('2022-01-01 00:00:00')  # Adjust this if different
test_data_x['t_1h'] = (test_data_x['t_1h'] - ref_time).dt.total_seconds() / 3600.0

# Convert test data to tensor
test_features = torch.tensor(test_data_x[['iu_ac', 't_1h', 'etat_barre']].values).float().to(device)

# Predict
model.eval()
with torch.no_grad():
    # test_predictions = model(test_features).flatten().cpu()
    test_predictions = model(test_features)

test_predictions
# print(len(test_data_x['iu_ac']))
# print(len(test_predictions.numpy()))

# Assuming lengths are now confirmed to match, proceed to create DataFrame
# if len(test_predictions):
#     predictions_df = pd.DataFrame({
#         'iu_ac': test_data_x['iu_ac'],
#         'predicted_q': test_predictions.numpy()
#     })
#     # Merge and compute MAE as before
#     result = pd.merge(baseline_data, predictions_df, on='iu_ac')
#     mae = torch.nn.functional.l1_loss(torch.tensor(result['predicted_q'].values).float(),
#                                       torch.tensor(result['q'].values).float())
#     print(f'Mean Absolute Error: {mae.item()}')
# else:
#     print("Error: Mismatch in data lengths.")
